In [1]:
import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
import datetime
import random
from scipy import stats
import math

#Avoiding Type 3 fonts in matplotlib plots
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.family'] = 'Arial'
pd.options.display.max_rows = 4000

In [2]:
font = {'size'   : 15}

matplotlib.rc('font', **font)
matplotlib.rc('lines', linewidth=2.0)
matplotlib.rc('lines', markersize=8)

# Modelling long-distance public trips
In this section we present the tools we used to derive the best-fitting amplified power-law process as well as the best-fitting truncated power-law for given emperical data-set

The parameters and values of the best-fitting processes are stored in a file. As the modelling procedure contains a parameter search this may take time to execute. For convenience, we also provided the `.pkl` files containing the results we obtained, allowing this section to be skipped.

## Error function

In order to determine whether a given CCDF $f$ describes the CCDF $g$ of our emperical data well, we use the following error function.
For a series of sample points $S$, it calculates the error as $\frac{2}{|S|}\sum_{x_i \in S} \dfrac{|f(x_i)-g(x_i)|}{ f(x_i)+g(x_i)}$.


In [3]:
''' Error function
Compute the distance between the CCDFs of two given data sets

d1,d2: the two input data sets, given as a 1-D list of samples.
Out of each input, a CCDF is generated. The CCDFs are then compared
according to above error metric.

max_d: is the empirical trip length value when the CCDF reaches 10^(-3) in a log-log scale CCDF plot, which the max range used for error calculation
num: is the number of sampling points in error calculation

return: error value
'''

def custom_distance(d1,d2,max_d,num):
    # Check input types
    if not (isinstance(d1, (np.ndarray, list, pd.Series)) and isinstance(d2, (np.ndarray, list, pd.Series))):
        raise ValueError('Wrong: Data type is not ndarray, List, or pd.Series')

    # Convert lists or pd.Series to numpy arrays for efficient computation
    d1 = np.array(d1)
    d2 = np.array(d2)

    # re-sampling
    # given the point x value and whole data, calculate the point of y value in CCDF
    sorted_d1 = np.sort(d1)
    y_d1=1- np.linspace(0, 1, len(sorted_d1), endpoint=False)

    sorted_d2 = np.sort(d2)
    y_d2=1- np.linspace(0, 1, len(sorted_d2), endpoint=False)
    
    # Create interpolation functions for the CCDFs
    interp_d1 = interp1d(sorted_d1, y_d1, bounds_error=False,fill_value=0)
    interp_d2 = interp1d(sorted_d2, y_d2, bounds_error=False,fill_value=0)

    
    # Find the overlapping range of the two distributions
    min_d = max(np.min(d1), np.min(d2))
    
    # Linearly spaced points for interpolation
    points = np.linspace(min_d, max_d, num)

    # Interpolate both distributions at the same points
    interp_y_d1 = interp_d1(points)
    interp_y_d2 = interp_d2(points)
    

    # Calculate the relative error in a vectorized way
    denom = (interp_y_d1 + interp_y_d2) / 2
    sum_error = np.abs(interp_y_d1 - interp_y_d2) / denom 
    error = np.nanmean(sum_error)
    
    return error


## Error function-----used for Modelling in the power law with the exponential truncation

In [4]:
''' New Error function-----used for find the power law with the exponential truncation
Compute the distance between the CCDFs of two given data sets

d1,d2: the two input data sets, given as a 1-D list of samples.
Out of each input, a CCDF is generated. The CCDFs are then compared
according to above error metric.

d1 is kind of CCDF value already(Fd), d2 is the regular 1-D samples
Ds is the distribution of points for getting d1, use the same range to get the interpolation value in d2
max_d: is the empirical trip length value when the CCDF reaches 10^(-3) in a log-log scale CCDF plot, which the max range used for error calculation
num: is the number of sampling points in error calculation

return: error value
'''

def custom_distance_exp(d1,d2,Ds,max_d,num):
    # Check input types
    if not (isinstance(d1, (np.ndarray, list, pd.Series)) and isinstance(d2, (np.ndarray, list, pd.Series))):
        raise ValueError('Wrong: Data type is not ndarray, List, or pd.Series')

    # Convert lists or pd.Series to numpy arrays for efficient computation
    d1 = np.array(d1)
    d2 = np.array(d2)

    sorted_d2 = np.sort(d2)
    y_d2=1- np.linspace(0, 1, len(sorted_d2), endpoint=False)

    # Create interpolation functions for the CCDFs,extended left=1, right=0
    interp_d1 = interp1d(Ds, d1, bounds_error=False,fill_value=(1,0)) # Ds is already sorted
    interp_d2 = interp1d(sorted_d2, y_d2, bounds_error=False,fill_value=(1,0))

    # Find the overlapping range of the two distributions
    min_d = max(np.min(Ds), np.min(d2))
    
    # Sampling method : Linear: # Linearly spaced points for interpolation
    points = np.linspace(min_d, max_d, num)

    # Interpolate both distributions at the same point
    interp_y_d1 = interp_d1(points)
    interp_y_d2 = interp_d2(points)


    # Calculate the relative error in a vectorized way
    denom = (interp_y_d1 + interp_y_d2) / 2
    sum_error = np.abs(interp_y_d1 - interp_y_d2) / denom 
    error = np.nanmean(sum_error)
        

    return error


## Get the maximum range of trip length for error cal

In [5]:
''' Calculate the trip length value (max_d in the error calculation function) when the CCDF reaches 10^(-3) in a log-log scale CCDF plot, 
    which will be used to determine the maximum range of error

d1: a 1-D dataset, like emipircal data in our project 
y_value: is the exact value in y axis in a log-log scale CCDF plot

return: the exact value (as possible) of corresponding x value ----> max_d
'''
def get_xCCDF(d1, y_value=1e-3):
    # cal ccdf
    d1 = np.array(d1)
    sorted_d1 = np.sort(d1)
    y_d1 = 1 - np.linspace(0, 1, len(sorted_d1), endpoint=False)
    
    # target（log-log scale, 10^-3）
    target_y = y_value
    
    # Find the first position where y_d1 <= target_y (y_d1 is monotonically decreasing)
    idx = np.searchsorted(y_d1[::-1], target_y, side='left') # searchsorted need monotonically increasing, so inverse the y_d1
    idx = len(y_d1) - idx  # inverse index
    
    # Extract neighbouring points (make sure y1 > target_y > y2)
    x1, x2 = sorted_d1[idx-1], sorted_d1[idx]
    y1, y2 = y_d1[idx-1], y_d1[idx]
    
    # Linear interpolation in log-log space
    log_x = np.log10(x1) + (np.log10(target_y) - np.log10(y1)) / (np.log10(y2) - np.log10(y1)) * (np.log10(x2) - np.log10(x1))
    x_target = 10 ** log_x
    
    return x_target


## Finding the best-fitting amplified power-law model for a given data-set
Here we derive the parameters of the amplified power-law process that best describes the empirical data-set.
The values of the best-fitting process are then stored for later use.

### Function of finding optimal parameters of training model

In [6]:
'''
Find the best-fitting parameter values for a distance-amplified power-law process
that follows the rules specified in the two previous blocks.

Input:
    combinable-- an array of all parameter combinations to consider, error is null at the begining
    sampleSize: number of samples for finding the optimal parameters
    min_distance,max_distance: range of trip length to be considered
    maxdistance: used for dynamic method, usually equals to max_distance
    max_d: the max range of trip length in error calculation
    num: number of sampling points for error calculation
Return:
    full-filled combinale error
'''

def find_opt_params(combinable,sampleSize,min_distance,maxdistance,df_vec,max_d,num):
    tmp_max=maxdistance/2 # half of the maximum allowed distance for any trip
    n_samples = len(combinable)

    for i in range(n_samples):
        # if i%100==0:
        #     print("Samples completed: ",i)
        #     print ("Current date and time : ",datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        eps, p, alp, error, all_dist = combinable[i]
        dist = []
       
        # simulate each trip
        for n in range(sampleSize):
            distance=9999999
            tmp=np.random.uniform(0,1)
            max_distance=np.sqrt(1-tmp)*tmp_max+tmp_max
            while(distance>=max_distance or distance<min_distance):

                # draw distance from the power-law distribution with parameter alp
                # amplification part
                #start with drawn power-law distance
                distance=np.random.uniform(0,1)**(-1/(alp-1))
                #compute how many times an amplifications occurs
                number=np.random.geometric(1-p)
                #amplify by factor (1+eps) each time
                distance=((1+eps)**(number-1))*distance

            dist.append(distance)

        combinable[i][4]=dist
        # error computation
        error=custom_distance(dist,df_vec,max_d,num)
        combinable[i][3]=error

    
    return combinable

## Power law with exponential truncation
- $f(d)=C*\dfrac{1}{d^{\alpha}} * e^{-\frac{d}{\gamma}}$

- $\sum_{d=100}^{900}f(d)=1$

$C=\dfrac{1}{\sum_{d=100}^{900}\dfrac{1}{d^{\alpha}} * e^{-\frac{d}{\gamma}}}$

- $\alpha=1.02....1.2$, step=0.01

- $\gamma=200,250,300....600$, step=50

- $d=100,150...900....2000$ step=50

- $\sum_{i=d}^{900}f(i)=F(d)$

Compute C and plot F(d), f(x) is kind of pdf
Known: gamma (interval can be 10 or 50firstly), alpha, d(distance from 100 to 900 )

$\sum_{d=1}^{10^3} 1/d^{1.02} * e^{-d/600}$

In [7]:
'''
Density function of the truncated power-law distribution
'''
def funcD(C,d,alpha,gamma):
    fD=C/d**alpha*(math.exp(-d/gamma))
    return fD

# This function is used as a helper function when computing C
def funcD_noC(d,alpha,gamma):
    fD_noC=1/d**alpha*(math.exp(-d/gamma))
    return fD_noC

In [8]:
'''
Finding optimal combination of parameters for power law with exponential truncation
'''
def get_results_PL_exp(alphas,gammas,Ds,data,max_d,num):
    results=[] # results contain alpha, gamma,C, error between the truncated power-law and emiprical data

    # for each combination of alpha and gamma
    for alpha in alphas:
        for gamma in gammas:
            # first compute the normalization constant C
            temp=0
            for d in Ds:
                temp+=funcD_noC(d,alpha,gamma)
            C=1/temp
    
            #compute the (approximate) CCDF F(d) of a truncated power-law
            #at several sample points given by Ds
            Fd=[0]*len(Ds)
            for idx,d in enumerate(Ds):
                for i in Ds[Ds>=d]:
                    Fd[idx]+=funcD(C,i,alpha,gamma)
            
            empircal_data=data.tolist()
            error=custom_distance_exp(Fd,empircal_data,Ds,max_d,num)         
            # error=ccdf_sampling_error(df_vec,Ds,Fd)
            results.append([alpha,gamma,C,error])

    #convert results to dataframe
    results_truncated=pd.DataFrame(results, columns =['alpha', 'gamma', 'C','error']) 
    # save the optimal results
    opt_truncated=results_truncated.iloc[results_truncated.error.argmin(),:]

    return opt_truncated